In [112]:
from dotenv import load_dotenv
import os
# Load the .env file
load_dotenv("../.env")

True

In [113]:
from typing import Annotated

from langchain_experimental.tools import PythonREPLTool
from langchain_openai import ChatOpenAI

# This executes code locally, which can be unsafe
python_repl_tool = PythonREPLTool()
llm = ChatOpenAI(model="gpt-4-turbo")

In [114]:
from langchain.agents import AgentExecutor, create_openai_tools_agent
from langchain_core.messages import BaseMessage, HumanMessage
from langchain_community.chat_models import ChatOllama
from langchain_experimental.llms.ollama_functions import OllamaFunctions
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_openai import ChatOpenAI
from langgraph.prebuilt import create_react_agent
from langchain_core.tools import BaseTool, tool
from langchain_core.runnables import Runnable

def create_agent(llm: ChatOpenAI, tools, system_prompt: str):
    prompt = ChatPromptTemplate.from_messages(
        [
            (
                "system",
                system_prompt,
            ),
            MessagesPlaceholder(variable_name="messages"),
        ]
    )
    agent = create_react_agent(llm, tools, state_modifier=prompt)
    return agent

In [124]:
from langchain_core.output_parsers.openai_functions import JsonOutputFunctionsParser
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.output_parsers import JsonOutputParser

members = ["Instructor", "Row Coder", "Column Coder"]
system_prompt = (
    "You are a supervisor tasked with managing a conversation between the"
    " following workers:  {members}. Given the following user request,"
    " respond with the worker to act next. Each worker will perform a"
    " task and respond with their results and status. When finished,"
    " respond with FINISH."
)
# Our team supervisor is an LLM node. It just picks the next agent to process
# and decides when the work is completed
options = ["FINISH"] + members
# Using openai function calling can make output parsing easier for us
function_def = {
    "name": "route",
    "description": "Select the next role.",
    "parameters": {
        "title": "routeSchema",
        "type": "object",
        "properties": {
            "next": {
                "title": "Next",
                "anyOf": [
                    {"enum": options},
                ],
            }
        },
        "required": ["next"],
    },
}
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        MessagesPlaceholder(variable_name="messages"),
        (
            "system",
            "Given the conversation above, who should act next?"
            " Or should we FINISH? Select one of: {options}",
        ),
    ]
).partial(options=str(options), members=", ".join(members))

llm = ChatOpenAI(model="gpt-4o-mini")

supervisor_chain = (
    prompt
    | llm.bind_tools(tools=[function_def], tool_choice="required")
)

def supervisor_agent(state):
    response = supervisor_chain.invoke(state)
    answer = response.tool_calls[0]["args"]["next"]
    state["next"] = answer
    print(answer + " is next.")
    # state["messages"] = HumanMessage(content="Next")
    return state

In [125]:
import pandas as pd
scramble = pd.read_csv("scrambled.csv",index_col=0)
truth = pd.read_csv("truth.csv", index_col=0)

# scramble_string = scramble.to_string()
# truth_string = truth.to_string()
# temp = "I have changed two dataframes, called scramble and truth, to strings. I want you to convert them back to dataframes "
# "and them change scramble to resemble truth as closely as possible without changing the quantitative information within. Do not "
# "change truth, only scramble. The two dataframes are truth and scramble as follows: " + scramble_string + " and " + truth_string + "."
# temp2 = "Given a string of a dataframe, convert it back into a dataframe and them generate another row before returning the string representation of your new dataframe. Use this string: "+ scramble_string + "."

@tool(parse_docstring=True)
def swap_scramble_column(column1: str, column2: str):
    """Swap two columns in the scramble dataframe.

    Args:
        column1: The name of the first column to swap.
        column2: The name of the second column to swap.
    """
    scramble[column1], scramble[column2] = scramble[column2].copy(), scramble[column1].copy()
    return scramble.head(1).to_string()

@tool(parse_docstring=True)
def change_scramble_column(old_column_name: str, new_column_name: str):
    """Change the name of a column in the scramble dataframe.

    Args:
        old_column_name: The name of the column to change.
        new_column_name: The new name for the column.
    """
    scramble.rename(columns = {old_column_name : new_column_name}, inplace = True)
    return scramble.head(1).to_string()

@tool(parse_docstring=True)
def change_scramble_row(old_name: str, new_name: str) -> str:
    """Change the name of a row in the scramble dataframe.

    Args:
        old_name: The index of the row to change.
        new_name: The new name for the row.
    """
    scramble.rename(index = {old_name : new_name}, inplace = True)
    return new_name + " updated."

# @tool(parse_docstring=True)
# def reindex_scramble_row(new_index: list[str]):
#     """Reorder the index of the scramble dataframe.

#     Args:
#         new_index: A list of the new index order. Pay attention to existing index values.
#     """
#     for index in new_index:
#         if index not in scramble.index:
#             new_index.remove(index)
#     scramble.reindex(new_index, inplace = True)
#     return scramble.to_string()

@tool
def get_truth():
    """get the truth dataframe"""
    return truth.to_string()

@tool
def get_scramble():
    """get the scramble dataframe"""
    return scramble.to_string()
swap_scramble_column.args

{'column1': {'title': 'Column1',
  'description': 'The name of the first column to swap.',
  'type': 'string'},
 'column2': {'title': 'Column2',
  'description': 'The name of the second column to swap.',
  'type': 'string'}}

In [126]:
import functools
import operator
from typing import Sequence, TypedDict

from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

from langgraph.graph import END, StateGraph, START
from langgraph.graph.message import add_messages


# The agent state is the input to each node in the graph
class AgentState(TypedDict):
    # The annotation tells the graph that new messages will always
    # be added to the current states
    messages: Annotated[Sequence[BaseMessage], add_messages]
    # The 'next' field indicates where to route to next
    next: str
    query : str





In [127]:
instructor_agent = create_agent(llm, [get_scramble, get_truth], "You are instructor. Your workflow is as follows: get and output scramble and truth. Then output instructions according to some rules. You are only allowed to swap columns, rename rows, and change the name of a column at a time. Order the row coder and column coder to make specific changes. BE specific. Do NOT make any modifications yourself. Be skeptical of Coder. When you are satisfied with the scramble, respond with FINISH.")


In [128]:
row_code_agent = create_agent(
    llm,
    [change_scramble_row],
    "You are Row Coder. Your job is to change the rows of scramble according to the instructions of instructor. However, "
    "the only thing you are allowed to do is change the names of the rows. Do not output anything except NEXT when finished.",
)

In [129]:
col_code_agent = create_agent(
    llm,
    [change_scramble_column, swap_scramble_column],
    "You are Column Coder. Your job is to change the columns of scramble according to the instructions of instructor. You can only swap two columns or"
      "change the name of singular columns. Do not output anything except DONE when finished.",
)

In [130]:
workflow = StateGraph(AgentState)
workflow.add_node("Instructor", instructor_agent)
workflow.add_node("Row Coder", row_code_agent)
workflow.add_node("Column Coder", col_code_agent)
workflow.add_node("supervisor", supervisor_agent)

In [131]:
workflow.add_edge("Instructor", "supervisor")
workflow.add_edge("Row Coder", "Column Coder")
workflow.add_edge("Column Coder", "supervisor")
# The supervisor populates the "next" field in the graph state
# which routes to a node or finishes
conditional_map = {k: k for k in members}
conditional_map["FINISH"] = END
workflow.add_conditional_edges("supervisor", lambda x: x["next"], conditional_map)
# Finally, add entrypoint
workflow.add_edge(START, "supervisor")
graph = workflow.compile()

In [132]:
from langchain_core.messages import HumanMessage
base_query = "I have changed two dataframes, called scramble and truth. I want you to  "
"change scramble to resemble truth as closely as possible without changing the quantitative information within. Instructor can give you both scramble and truth as strings. If coder sends something, let instructor check coder's work."
input = {
        "messages": [
            HumanMessage(content=base_query),
        ],
        "query" : base_query
    }
l = 0
for output in graph.stream(input, stream_mode="values", config={"recursion_limit": 40}):
    # stream() yields dictionaries with output keyed by node name
    messages = output["messages"]
    if len(messages) > l:
        for message in messages[l+1:]:
            message.pretty_print()
        l = len(messages)
    print("\n---\n")


---

Instructor is next.

---

================================= Tool Message =================================
Name: get_scramble

     Revenue  Profit  Quantity
P4      2294     443        24
P5      2130     591        71
P6      2095     513        71
P16     4444     847        82
P17     4171     956        48
P10     3169     376        64
P11     1466     260        73
P14     2482     121        16
P3      1860     869        89
P19     4735     574        13
P20     1130     158        98
P12     2238     559        12
P18     3919     660        27
P8      4092     485        71
P9      2638     291        60
P7      4772     905        56
P1      4174     761        51
P13     1330     413        60
P15     3135     352        30
================================= Tool Message =================================
Name: get_truth

            Sales  Profit  Units Sold
Product 1    4174     761          51
Product 2    4507     408          69
Product 3    1860     869          

In [133]:
scramble

,Sales,Profit,Units Sold
Product 4,24,2294,443
Product 5,71,2130,591
Product 6,71,2095,513
Product 16,82,4444,847
Product 17,48,4171,956
Product 10,64,3169,376
Product 11,73,1466,260
Product 14,16,2482,121
Product 3,89,1860,869
Product 19,13,4735,574
